In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import pandas as pd
from PIL import Image

DATA_DIR = './'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
LABELS_CSV = os.path.join(DATA_DIR, 'train_labels.csv')

# データセットクラス
class PCamDataset(Dataset):
    def __init__(self, img_dir, labels_csv, transform=None):
        self.img_dir = img_dir
        self.labels = pd.read_csv(labels_csv)
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_id = self.labels.iloc[idx, 0]
        label = self.labels.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, f"{img_id}.tif")
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

# 前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# データローダー
train_dataset = PCamDataset(TRAIN_IMG_DIR, LABELS_CSV, transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 簡単なCNNモデル
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 24 * 24, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 24 * 24)
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x.squeeze()

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)

# 損失関数と最適化
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習ループ（1エポック例）
for images, labels in train_loader:
    images = images.to(device)
    labels = labels.float().to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f'loss: {loss.item()}')
    break  # サンプルとして1バッチのみ

loss: 0.6939420104026794


In [6]:
import torchvision.models as models
from torchvision import transforms

TEST_IMG_DIR = os.path.join(DATA_DIR, 'test')

# ResNet50の転移学習モデル（出力1次元に変更）
resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 1)
model = resnet50.to(device)

# 前処理（ResNet50用、3ch・224x224）
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# データセット・ローダー（再定義）
train_dataset_resnet = PCamDataset(TRAIN_IMG_DIR, LABELS_CSV, transform_resnet)
train_loader_resnet = DataLoader(train_dataset_resnet, batch_size=32, shuffle=True)

# 損失関数と最適化
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 学習ループ（1バッチのみ例）
for images, labels in train_loader_resnet:
    images = images.to(device)
    labels = labels.float().to(device)
    optimizer.zero_grad()
    outputs = model(images).squeeze()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f'loss: {loss.item()}')
    break  # サンプルとして1バッチのみ

# 予測例（test画像の1枚）
test_img_name = os.listdir(TEST_IMG_DIR)[0]
test_img_path = os.path.join(TEST_IMG_DIR, test_img_name)
test_img = Image.open(test_img_path)
test_tensor = transform_resnet(test_img).unsqueeze(0).to(device)
model.eval()
with torch.no_grad():
    output = model(test_tensor)
    pred = torch.sigmoid(output).item()
    print(f'{test_img_name} の予測値: {pred:.4f}')

/home/gotou/miniconda3/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gotou/miniconda3/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loss: 0.6149500608444214
05dcf7099d054b8af930253bbc2c3fb9055ac0a5.tif の予測値: 0.3388


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

labels_df = pd.read_csv(LABELS_CSV)


class PCamDataset(Dataset):
    def __init__(self, img_dir, labels_df, transform=None):
        self.img_dir = img_dir
        self.labels = labels_df
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_id = self.labels.iloc[idx, 0]
        label = self.labels.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, f"{img_id}.tif")
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# train/val分割
train_df, val_df = train_test_split(labels_df, test_size=0.1, random_state=42, stratify=labels_df['label'])

train_dataset = PCamDataset(TRAIN_IMG_DIR, train_df, transform_resnet)
val_dataset = PCamDataset(TRAIN_IMG_DIR, val_df, transform_resnet)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# ResNet50モデル
resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 1)
model = resnet50.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 学習ループ
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # 検証
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.float().to(device)
            outputs = model(images).squeeze()
            preds = torch.sigmoid(outputs) > 0.5
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    val_acc = accuracy_score(val_labels, val_preds)
    print(f'Val Accuracy: {val_acc*100:.2f}%')

# テストデータ予測
test_img_names = os.listdir(TEST_IMG_DIR)
test_preds = []
model.eval()
for img_name in test_img_names:
    img_path = os.path.join(TEST_IMG_DIR, img_name)
    img = Image.open(img_path)
    tensor = transform_resnet(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(tensor)
        pred = torch.sigmoid(output).item()
        test_preds.append(pred)
print('Test予測例:', test_preds[:5])

/home/gotou/miniconda3/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gotou/miniconda3/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 0.1443
Val Accuracy: 96.72%
Epoch 2/5, Loss: 0.0883
Val Accuracy: 95.65%
Epoch 3/5, Loss: 0.0633
Val Accuracy: 97.38%
Epoch 4/5, Loss: 0.0444
Val Accuracy: 97.35%
Epoch 5/5, Loss: 0.0331
Val Accuracy: 97.15%
Test予測例: [0.9999980926513672, 2.3613171151737333e-07, 0.0028101117350161076, 0.7870376706123352, 0.002361058723181486]


In [9]:
import pandas as pd

# 予測結果をCSVに保存
submission = pd.DataFrame({
    'id': [os.path.splitext(name)[0] for name in test_img_names],
    'label': test_preds
})
submission.to_csv('submission.csv', index=False)
print('submission.csvを保存しました')

submission.csvを保存しました
